## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 

Sarving and Death is taken care of elsewhere, so this notebook won't have that filter. 


In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "..")
sys.path.insert(0, "../../functions/popgrowth/")

from app.connectors import cmdb_graph
from app.objects import time
from app.objects import population
from helpers import test_queries

c = cmdb_graph.CosmosdbClient()



executing local windows deployment


In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [4]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2023-07-23T14:51:07.018856+00:00 UTU:28311 >

In [5]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28311}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [6]:
params['pop_health_requirement'] = 0.4

In [7]:
# from tools import growth
global_health_manager = population.Global_Pop_Manager(params,c)


In [8]:
global_health_manager.get_pop_health()

pops_df = pd.DataFrame([d['pop'] for d in global_health_manager.data])
species_df = pd.DataFrame([d['species'] for d in global_health_manager.data])
locations_df = pd.DataFrame([d['location'] for d in global_health_manager.data])
factions_df = pd.DataFrame([d['faction'] for d in global_health_manager.data])

In [9]:
# pops_df,species_df,locations_df,factions_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


health isIdle                  name          objid  conformity  literacy  \
0      0.70  false         Obattla Magor  7467985401518       0.421     0.376   
1      0.70   true            Obattla Pe  5100638967625       0.325     0.352   
2      0.70   true           Obattla Van  1800951322768       0.446     0.390   
3      0.70   true         Obattla Rochi  0692394121482       0.452     0.395   
4      0.70   true            Hassia Sin  0248671359952       0.623     0.215   
..      ...    ...                   ...            ...         ...       ...   
121    0.42   true    Serzeampro Bilnoru  9142883900818       0.557     0.544   
122    0.42   true              Ciki Yqa  2819449749992       0.298     0.586   
123    0.42   true      Serzeampro Wilfo  0872830607503       0.469     0.545   
124    0.42   true  Serzeampro Gradonnue  7834145976449       0.340     0.495   
125    0.42   true            Ciki Fesha  0652360161340       0.524     0.726   

     aggression  constitution           isIn  industry  wealth  \
0         0.632         0.338  8822366435138    0.4850  0.4305   
1         0.606         0.450  8822366435138    0.5280  0.4400   
2         0.549         0.573  8822366435138    0.5610  0.4755   
3         0.514         0.546  8822366435138    0.5300  0.4625   
4         0.704         0.569  4146242421204    0.6365  0.4258   
..          ...           ...            ...       ...     ...   
121       0.609         0.557  4873232607770    0.3500  0.3900   
122       0.489         0.468  0015468195520    0.2660  0.2940   
123       0.722         0.570  4873232607770    0.3940  0.2740   
124       0.532         0.635  4873232607770    0.3450  0.3030   
125       0.452         0.635  0015468195520    0.2970  0.3150   

     factionLoyalty       username objtype             id  
0             0.505  BillmanLocal2     pop  7467985401518  
1             0.459  BillmanLocal2     pop  5100638967625  
2             0.423  BillmanLocal2     pop  1800951322768  
3             0.354  BillmanLocal2     pop  0692394121482  
4             0.662  BillmanLocal2     pop  0248671359952  
..              ...            ...     ...            ...  
121           0.749  Billmanserver     pop  9142883900818  
122           0.316  Billmanserver     pop  2819449749992  
123           0.494  Billmanserver     pop  0872830607503  
124           0.331  Billmanserver     pop  7834145976449  
125           0.521  Billmanserver     pop  0652360161340  

[126 rows x 15 columns]

In [10]:
species_df

name          objid   consumes                    effuses  \
0    Koykudra  4291284873310  [Organic]  [Organic waste, Plastics]   
1    Koykudra  4291284873310  [Organic]  [Organic waste, Plastics]   
2    Koykudra  4291284873310  [Organic]  [Organic waste, Plastics]   
3    Koykudra  4291284873310  [Organic]  [Organic waste, Plastics]   
4    Koykudra  4291284873310  [Organic]  [Organic waste, Plastics]   
..        ...            ...        ...                        ...   
121       Til  5478087080681  [Organic]  [Organic waste, Plastics]   
122       Til  5478087080681  [Organic]  [Organic waste, Plastics]   
123       Til  5478087080681  [Organic]  [Organic waste, Plastics]   
124       Til  5478087080681  [Organic]  [Organic waste, Plastics]   
125       Til  5478087080681  [Organic]  [Organic waste, Plastics]   

     viral_resilience  habitat_resilience       username  objtype  \
0                 0.7                 0.2  BillmanLocal2  species   
1                 0.7                 0.2  BillmanLocal2  species   
2                 0.7                 0.2  BillmanLocal2  species   
3                 0.7                 0.2  BillmanLocal2  species   
4                 0.7                 0.2  BillmanLocal2  species   
..                ...                 ...            ...      ...   
121               0.7                 0.2  Billmanserver  species   
122               0.7                 0.2  Billmanserver  species   
123               0.7                 0.2  Billmanserver  species   
124               0.7                 0.2  Billmanserver  species   
125               0.7                 0.2  Billmanserver  species   

                id  
0    4291284873310  
1    4291284873310  
2    4291284873310  
3    4291284873310  
4    4291284873310  
..             ...  
121  5478087080681  
122  5478087080681  
123  5478087080681  
124  5478087080681  
125  5478087080681  

[126 rows x 9 columns]

In [11]:
locations_df

name        class          objid  radius   mass  orbitsDistance  \
0     Dorf  terrestrial  2972908340606   0.884  0.882           1.057   
1     Dorf  terrestrial  2972908340606   0.884  0.882           1.057   
2     Dorf  terrestrial  2972908340606   0.884  0.882           1.057   
3     Dorf  terrestrial  2972908340606   0.884  0.882           1.057   
4     Dorf  terrestrial  2972908340606   0.884  0.882           1.057   
..     ...          ...            ...     ...    ...             ...   
121  Taipe  terrestrial  8490894854096   0.898  0.113           1.087   
122  Taipe  terrestrial  8490894854096   0.898  0.113           1.087   
123  Taipe  terrestrial  8490894854096   0.898  0.113           1.087   
124  Taipe  terrestrial  8490894854096   0.898  0.113           1.087   
125  Taipe  terrestrial  8490894854096   0.898  0.113           1.087   

          orbitsId orbitsName isSupportsLife isPopulated isHomeworld  \
0    9570931993836   Lisallos           true        true        true   
1    9570931993836   Lisallos           true        true        true   
2    9570931993836   Lisallos           true        true        true   
3    9570931993836   Lisallos           true        true        true   
4    9570931993836   Lisallos           true        true        true   
..             ...        ...            ...         ...         ...   
121  1849212388331   Lindraja           true        true        true   
122  1849212388331   Lindraja           true        true        true   
123  1849212388331   Lindraja           true        true        true   
124  1849212388331   Lindraja           true        true        true   
125  1849212388331   Lindraja           true        true        true   

          username objtype             id  
0    BillmanLocal2  planet  2972908340606  
1    BillmanLocal2  planet  2972908340606  
2    BillmanLocal2  planet  2972908340606  
3    BillmanLocal2  planet  2972908340606  
4    BillmanLocal2  planet  2972908340606  
..             ...     ...            ...  
121  Billmanserver  planet  8490894854096  
122  Billmanserver  planet  8490894854096  
123  Billmanserver  planet  8490894854096  
124  Billmanserver  planet  8490894854096  
125  Billmanserver  planet  8490894854096  

[126 rows x 14 columns]

In [12]:
factions_df

pop_locations        name  \
0    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...     Obattla   
1    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...     Obattla   
2    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...     Obattla   
3    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...     Obattla   
4    [[0, 0], [-2, 2], [-1, -1], [2, 0], [-1, 3], [...      Hassia   
..                                                 ...         ...   
121  [[0, 0], [2, -2], [0, -2], [-2, -2], [0, 2], [...  Serzeampro   
122  [[0, 0], [0, -2], [-2, 2], [-2, -2], [2, 2], [...        Ciki   
123  [[0, 0], [2, -2], [0, -2], [-2, -2], [0, 2], [...  Serzeampro   
124  [[0, 0], [2, -2], [0, -2], [-2, -2], [0, 2], [...  Serzeampro   
125  [[0, 0], [0, -2], [-2, 2], [-2, -2], [2, 2], [...        Ciki   

             objid    lat  long       username  objtype             id  \
0    8822366435138 -0.149   0.0  BillmanLocal2  faction  8822366435138   
1    8822366435138 -0.149   0.0  BillmanLocal2  faction  8822366435138   
2    8822366435138 -0.149   0.0  BillmanLocal2  faction  8822366435138   
3    8822366435138 -0.149   0.0  BillmanLocal2  faction  8822366435138   
4    4146242421204  0.149   0.0  BillmanLocal2  faction  4146242421204   
..             ...    ...   ...            ...      ...            ...   
121  4873232607770  0.113   0.0  Billmanserver  faction  4873232607770   
122  0015468195520 -0.113   0.0  Billmanserver  faction  0015468195520   
123  4873232607770  0.113   0.0  Billmanserver  faction  4873232607770   
124  4873232607770  0.113   0.0  Billmanserver  faction  4873232607770   
125  0015468195520 -0.113   0.0  Billmanserver  faction  0015468195520   

    pop_loactions  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
..            ...  
121           NaN  
122           NaN  
123           NaN  
124           NaN  
125           NaN  

[126 rows x 9 columns]

In [13]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
pops_df

health isIdle                  name          objid  conformity  literacy  \
0      0.70  false         Obattla Magor  7467985401518       0.421     0.376   
1      0.70   true            Obattla Pe  5100638967625       0.325     0.352   
2      0.70   true           Obattla Van  1800951322768       0.446     0.390   
3      0.70   true         Obattla Rochi  0692394121482       0.452     0.395   
4      0.70   true            Hassia Sin  0248671359952       0.623     0.215   
..      ...    ...                   ...            ...         ...       ...   
121    0.42   true    Serzeampro Bilnoru  9142883900818       0.557     0.544   
122    0.42   true              Ciki Yqa  2819449749992       0.298     0.586   
123    0.42   true      Serzeampro Wilfo  0872830607503       0.469     0.545   
124    0.42   true  Serzeampro Gradonnue  7834145976449       0.340     0.495   
125    0.42   true            Ciki Fesha  0652360161340       0.524     0.726   

     aggression  constitution           isIn  industry  wealth  \
0         0.632         0.338  8822366435138    0.4850  0.4305   
1         0.606         0.450  8822366435138    0.5280  0.4400   
2         0.549         0.573  8822366435138    0.5610  0.4755   
3         0.514         0.546  8822366435138    0.5300  0.4625   
4         0.704         0.569  4146242421204    0.6365  0.4258   
..          ...           ...            ...       ...     ...   
121       0.609         0.557  4873232607770    0.3500  0.3900   
122       0.489         0.468  0015468195520    0.2660  0.2940   
123       0.722         0.570  4873232607770    0.3940  0.2740   
124       0.532         0.635  4873232607770    0.3450  0.3030   
125       0.452         0.635  0015468195520    0.2970  0.3150   

     factionLoyalty       username objtype             id      roll   grow  
0             0.505  BillmanLocal2     pop  7467985401518  0.325517   True  
1             0.459  BillmanLocal2     pop  5100638967625  0.705648  False  
2             0.423  BillmanLocal2     pop  1800951322768  0.608676  False  
3             0.354  BillmanLocal2     pop  0692394121482  0.723503  False  
4             0.662  BillmanLocal2     pop  0248671359952  0.552915   True  
..              ...            ...     ...            ...       ...    ...  
121           0.749  Billmanserver     pop  9142883900818  0.455465  False  
122           0.316  Billmanserver     pop  2819449749992  0.850461  False  
123           0.494  Billmanserver     pop  0872830607503  0.486702  False  
124           0.331  Billmanserver     pop  7834145976449  0.170045   True  
125           0.521  Billmanserver     pop  0652360161340  0.300788   True  

[126 rows x 17 columns]

In [14]:
print(f"{len(reproducing_pops)} of {len(pops_df)} will grow")
reproducing_pops

41 of 126 will grow


health isIdle                      name          objid  conformity  \
0      0.70  false             Obattla Magor  7467985401518       0.421   
4      0.70   true                Hassia Sin  0248671359952       0.623   
5      0.70   true             Obattla Artes  9819392335838       0.488   
7      0.42   true          Obattla Magordra  0126014006101       0.421   
9      0.42   true        Obattla Arteskreis  4877750744555       0.509   
10     0.42   true            Obattla Sertre  3644546786855       0.325   
18     0.70   true      Zopon Haydorfkerberg  1417654942596       0.370   
22     0.70   true           Zopon Lischarha  2152501246930       0.554   
23     0.70  false            Tilcasport Tla  3298484338584       0.780   
24     0.70   true    Tilcasport Boupatciace  4742850136180       0.692   
25     0.70   true      Tilcasport Siatiapec  5669843388137       0.859   
26     0.75   true          Tilcasport Niaar  7036253430140       0.584   
27     0.75   true       Tilcasport Tankyque  8226274126289       0.705   
28     0.75   true            Tilcasport Tos  1437057417022       0.752   
29     0.70   true          Tilcasport Enwil  3440955961419       0.683   
34     0.42   true         Zopon Lischarhage  2935089853375       0.558   
36     0.42   true          Tilcasport Toski  1537196746887       0.814   
40     0.42   true        Obattla Rochifield  1059188390521       0.364   
41     0.42   true          Obattla Artesner  3135645518769       0.473   
44     0.42   true             Ganjtil Woque  0002436592279       0.425   
48     0.42   true  Tilcasport Boupatciaceou  6532454518591       0.646   
67     0.42   true            Obattla Vannio  5064054715259       0.491   
68     0.42   true          Obattla Rochihen  3954554316101       0.404   
69     0.42   true   Zopon Haydorfkerbergbou  4099521721838       0.370   
72     0.42   true        Zopon Lischarhaner  5581254450725       0.468   
75     0.42   true          Obattla Magorkon  7511884158024       0.417   
78     0.42   true          Obattla Artestai  3735917286448       0.416   
79     0.42   true            Ganjtil Wohill  9507225996006       0.422   
88     0.45   true        Tilcasport Tosrang  9820062295194       0.720   
89     0.42   true          Obattla Magordel  1206973395832       0.436   
90     0.42   true            Obattla Vanpla  7366041268748       0.378   
94     0.42   true   Zopon Haydorfkerbergkay  9033250037782       0.419   
96     0.42   true   Ganjtil Lismontpastilmi  2430051459595       0.538   
100    0.70   true          Serzeampro Bilno  9890591392190       0.481   
104    0.70   true                Ciki Basma  1247897538416       0.510   
105    0.70   true         Serzeampro Gradon  8096543483724       0.432   
108    0.42   true           Obattla Rochizu  3992306323224       0.505   
118    0.45   true     Tilcasport Tankyquepu  7271925834872       0.798   
120    0.42   true      Tilcasport Enwilburg  1908164461185       0.782   
124    0.42   true      Serzeampro Gradonnue  7834145976449       0.340   
125    0.42   true                Ciki Fesha  0652360161340       0.524   

     literacy  aggression  constitution           isIn  industry  wealth  \
0       0.376       0.632         0.338  8822366435138    0.4850  0.4305   
4       0.215       0.704         0.569  4146242421204    0.6365  0.4258   
5       0.383       0.572         0.570  8822366435138    0.5710  0.4770   
7       0.474       0.684         0.377  8822366435138    0.2910  0.2290   
9       0.350       0.660         0.488  8822366435138    0.3430  0.1960   
10      0.441       0.572         0.532  8822366435138    0.3470  0.3470   
18      0.389       0.509         0.389  9376731217333    0.4490  0.4190   
22      0.508       0.614         0.356  9376731217333    0.4850  0.4965   
23      0.751       0.252         0.195  4232678858631    0.2235  0.4872   
24      0.822       0.374         0.187  4232678858631    0.2555  0.4888   
25      0.824     

Going to create species objects for the populations that will reproduce

In [15]:
# population.species.Species()
reproducing_species = species_df.loc[reproducing_pops.index].drop_duplicates()
reproducing_species

name          objid   consumes                    effuses  \
0    Koykudra  4291284873310  [Organic]  [Organic waste, Plastics]   
18         Si  7324266402745  [Organic]  [Organic waste, Plastics]   
23     Towner  1245644824657  [Organic]  [Organic waste, Plastics]   
100       Til  5478087080681  [Organic]  [Organic waste, Plastics]   

     viral_resilience  habitat_resilience                           username  \
0                 0.7                 0.2                      BillmanLocal2   
18                0.7                 0.2                          manderson   
23                0.7                 0.2  william.jeffrey.harding@gmail.com   
100               0.7                 0.2                      Billmanserver   

     objtype             id  
0    species  4291284873310  
18   species  7324266402745  
23   species  1245644824657  
100  species  5478087080681

In [16]:
species = [population.species.Species(reproducing_species.T.to_dict()[i]) for i in  reproducing_species.T.to_dict().keys()]
species

[<species: None; 4291284873310; Koykudra>,
 <species: None; 7324266402745; Si>,
 <species: None; 1245644824657; Towner>,
 <species: None; 5478087080681; Til>]

In [17]:
pops_df

health isIdle                  name          objid  conformity  literacy  \
0      0.70  false         Obattla Magor  7467985401518       0.421     0.376   
1      0.70   true            Obattla Pe  5100638967625       0.325     0.352   
2      0.70   true           Obattla Van  1800951322768       0.446     0.390   
3      0.70   true         Obattla Rochi  0692394121482       0.452     0.395   
4      0.70   true            Hassia Sin  0248671359952       0.623     0.215   
..      ...    ...                   ...            ...         ...       ...   
121    0.42   true    Serzeampro Bilnoru  9142883900818       0.557     0.544   
122    0.42   true              Ciki Yqa  2819449749992       0.298     0.586   
123    0.42   true      Serzeampro Wilfo  0872830607503       0.469     0.545   
124    0.42   true  Serzeampro Gradonnue  7834145976449       0.340     0.495   
125    0.42   true            Ciki Fesha  0652360161340       0.524     0.726   

     aggression  constitution           isIn  industry  wealth  \
0         0.632         0.338  8822366435138    0.4850  0.4305   
1         0.606         0.450  8822366435138    0.5280  0.4400   
2         0.549         0.573  8822366435138    0.5610  0.4755   
3         0.514         0.546  8822366435138    0.5300  0.4625   
4         0.704         0.569  4146242421204    0.6365  0.4258   
..          ...           ...            ...       ...     ...   
121       0.609         0.557  4873232607770    0.3500  0.3900   
122       0.489         0.468  0015468195520    0.2660  0.2940   
123       0.722         0.570  4873232607770    0.3940  0.2740   
124       0.532         0.635  4873232607770    0.3450  0.3030   
125       0.452         0.635  0015468195520    0.2970  0.3150   

     factionLoyalty       username objtype             id      roll   grow  
0             0.505  BillmanLocal2     pop  7467985401518  0.325517   True  
1             0.459  BillmanLocal2     pop  5100638967625  0.705648  False  
2             0.423  BillmanLocal2     pop  1800951322768  0.608676  False  
3             0.354  BillmanLocal2     pop  0692394121482  0.723503  False  
4             0.662  BillmanLocal2     pop  0248671359952  0.552915   True  
..              ...            ...     ...            ...       ...    ...  
121           0.749  Billmanserver     pop  9142883900818  0.455465  False  
122           0.316  Billmanserver     pop  2819449749992  0.850461  False  
123           0.494  Billmanserver     pop  0872830607503  0.486702  False  
124           0.331  Billmanserver     pop  7834145976449  0.170045   True  
125           0.521  Billmanserver     pop  0652360161340  0.300788   True  

[126 rows x 17 columns]

In [16]:

for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    f = factions_df.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    location = locations_df.loc[i].to_dict()
    child = growth.grow_pop(p,species,params,syllables)    
    pop_locations = growth.assign_pop_to_faction(f)    
    c.patch_property(f['objid'], 'pop_locations', str(pop_locations))
    nodes.append(child)
    event = growth.population_growth_event(p, location,child, params)
    nodes.append(event)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isIn"], "label": "isIn"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOf"})
    edges.append({"node1": child["objid"], "node2": location["objid"], "label": "inhabits"})
    event_edges.append(c.create_custom_edge(event,location,'happenedAt'))
    event_edges.append(c.create_custom_edge(p,event,'caused'))


In [17]:
f

{'pop_locations': '[[0, 0], [0, -2], [-2, 2], [-2, -2], [2, 2], [-1, 1], [0, 2], [-2, 0], [1, -1], [1, 3]]',
 'name': 'Ciki',
 'objid': '0015468195520',
 'lat': -0.113,
 'long': 0.0,
 'username': 'Billmanserver',
 'objtype': 'faction',
 'id': '0015468195520',
 'pop_loactions': nan}

In [18]:
pop_locations = growth.assign_pop_to_faction(f) 
pop_locations


[[0, 0],
 [0, -2],
 [-2, 2],
 [-2, -2],
 [2, 2],
 [-1, 1],
 [0, 2],
 [-2, 0],
 [1, -1],
 [1, 3],
 [-1, 3]]

In [19]:
c.patch_property(f['objid'], 'pop_locations', str(pop_locations))

In [20]:
c.run_query(f"""
g.V().has('objid','{f['objid']}').valueMap()
""")    

c.res

[{'pop_locations': ['[[0, 0], [0, -2], [-2, 2], [-2, -2], [2, 2], [-1, 1], [0, 2], [-2, 0], [1, -1], [1, 3], [-1, 3]]'],
  'name': ['Ciki'],
  'objid': ['0015468195520'],
  'lat': [-0.113],
  'long': [0.0],
  'username': ['Billmanserver'],
  'objtype': ['faction']}]

In [21]:
p

{'health': 0.7,
 'isIdle': 'true',
 'name': 'Ciki Fe',
 'objid': '8876287369940',
 'conformity': 0.456,
 'literacy': 0.725,
 'aggression': 0.428,
 'constitution': 0.562,
 'isIn': '0015468195520',
 'industry': 0.495,
 'wealth': 0.61,
 'factionLoyalty': 0.529,
 'username': 'Billmanserver',
 'objtype': 'pop',
 'id': '8876287369940'}

In [22]:
growth.grow_pop(p,species,params,syllables)

{'health': 0.42,
 'isIdle': 'true',
 'name': 'Ciki Fehas',
 'objid': '7005589919552',
 'conformity': 0.524,
 'literacy': 0.79,
 'aggression': 0.489,
 'constitution': 0.548,
 'isIn': '0015468195520',
 'industry': 0.297,
 'wealth': 0.418,
 'factionLoyalty': 0.504,
 'username': 'Billmanserver',
 'objtype': 'pop',
 'id': '7005589919552',
 'label': 'pop'}

In [23]:
pd.DataFrame.from_records(nodes)


health isIdle                        name          objid  conformity  \
0     0.42   true            Obattla Magorpro  0913584747812       0.520   
1      NaN    NaN           population growth  7184852353123         NaN   
2     0.42   true                Obattla Peby  0630745128207       0.401   
3      NaN    NaN           population growth  2196391002317         NaN   
4     0.42   true              Obattla Vangar  5675131856567       0.511   
5      NaN    NaN           population growth  7356076059018         NaN   
6     0.42   true                Hassia Sinmu  3307181878378       0.553   
7      NaN    NaN           population growth  2276175445082         NaN   
8     0.42   true            Obattla Arteswil  8738754631576       0.535   
9      NaN    NaN           population growth  5497131605023         NaN   
10    0.42   true          Obattla Sersprings  3727965196255       0.274   
11     NaN    NaN           population growth  0690887141373         NaN   
12    0.42   true        Ganjtil Siacarratgor  2152499680520       0.609   
13     NaN    NaN           population growth  4840616715473         NaN   
14    0.42   true                Ganjtil Wogi  6064514068524       0.512   
15     NaN    NaN           population growth  4703077847671         NaN   
16    0.42   true     Zopon Haydorfkerbergpen  3216423906414       0.300   
17     NaN    NaN           population growth  0775878869326         NaN   
18    0.42   true               Ganjtil Aytuu  4814336118524       0.620   
19     NaN    NaN           population growth  7345988827345         NaN   
20    0.42   true      Ganjtil Quabanmarwarre  3175874154129       0.510   
21     NaN    NaN           population growth  6378902062013         NaN   
22    0.42   true         Zopon Lischarhapest  6827404859809       0.520   
23     NaN    NaN           population growth  8513510577003         NaN   
24    0.42   true  Tilcasport Boupatciacerich  6986617453979       0.737   
25     NaN    NaN           population growth  5260747328170         NaN   
26    0.42   true     Tilcasport Siatiapeckes  8646845240336       0.861   
27     NaN    NaN           population growth  7782388659825         NaN   
28    0.45   true         Tilcasport Niaarsin  1820268905401       0.503   
29     NaN    NaN           population growth  3200486583332         NaN   
30    0.42   true          Tilcasport Enwilby  5408205765317       0.726   
31     NaN    NaN           population growth  4526569155793         NaN   
32    0.42   true            Serzeampro Wilye  4271185084463       0.354   
33     NaN    NaN           population growth  2919916850958         NaN   
34    0.42   true               Ciki Basmadia  4667958701560       0.511   
35     NaN    NaN           population growth  4183534367737         NaN   
36    0.42   true        Serzeampro Gradonnor  2901583468148       0.502   
37     NaN    NaN           population growth  4606339513308         NaN   
38    0.42   true                   Ciki Fepe  6707630942353       0.407   
39     NaN    NaN           population growth  6276059687128         NaN   

    literacy  aggression  constitution           isIn  industry  wealth  \
0      0.304       0.595         0.260  8822366435138     0.291   0.328   
1        NaN         NaN           NaN            NaN       NaN     NaN   
2      0.323       0.666         0.380  8822366435138     0.317   0.291   
3        NaN         NaN           NaN            NaN       NaN     NaN   
4      0.404       0.519         0.473  8822366435138     0.337   0.349   
5        NaN         NaN           NaN            NaN       NaN     NaN   
6      0.175       0.605         0.633  4146242421204     0.382   0.201   
7        NaN         NaN           NaN            NaN       NaN     NaN   
8      0.391       0.591         0.642  8822366435138     0.343   0.378   
9        NaN         NaN           NaN            NaN       NaN     NaN   
10     0.438       0.628         0.522  8822366435138     0.34

In [24]:
pd.DataFrame.from_records(edges)

node1          node2     label
0   0913584747812  7467985401518   childOf
1   0913584747812  8822366435138      isIn
2   0913584747812  4291284873310      isOf
3   0913584747812  2972908340606  inhabits
4   0630745128207  5100638967625   childOf
..            ...            ...       ...
75  2901583468148  8490894854096  inhabits
76  6707630942353  8876287369940   childOf
77  6707630942353  0015468195520      isIn
78  6707630942353  5478087080681      isOf
79  6707630942353  8490894854096  inhabits

[80 rows x 3 columns]

In [25]:
event_edges

["\n        g.V().has('objid','7184852353123')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2972908340606'))\n        ",
 "\n        g.V().has('objid','7467985401518')\n            .addE('caused')\n            .to(g.V().has('objid','7184852353123'))\n        ",
 "\n        g.V().has('objid','2196391002317')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2972908340606'))\n        ",
 "\n        g.V().has('objid','5100638967625')\n            .addE('caused')\n            .to(g.V().has('objid','2196391002317'))\n        ",
 "\n        g.V().has('objid','7356076059018')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2972908340606'))\n        ",
 "\n        g.V().has('objid','1800951322768')\n            .addE('caused')\n            .to(g.V().has('objid','7356076059018'))\n        ",
 "\n        g.V().has('objid','2276175445082')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2972908340606'))\n        

In [26]:
growth.grow(c,params,syllables)

In [ ]:
c.upload_data()